# Stimare Q function ottimale
## 1. Quanto tempo ci mette per convergere a Q*, al variare di tau
## 2. Distanza tra Q* e Q_t 
## 3. distanza tra Q_0 e Q_0 appresa tramite i valori di tau
## 4. cercare di integrare i bound trovati

### Estrarre policy da Q
### Confrontare due diverse Q valutando diverse metriche 
### Calcolare Q*
### Tentativo di implementazione di un curriculum
### Plot functions

In [8]:
import numpy as np
from TMDP import TMDP
from river_swim import River

from algorithms import Q_learning, eps_greedy, SARSA, get_policy


In [9]:
# Test with tau=0.9
gamma = 0.9
river = River(gamma)
tau = 0.9
xi = np.ones(river.nS)*1/river.nS
tmdp = TMDP(river, xi, tau, gamma)


In [18]:
s = tmdp.reset()
M = 1000
Q = np.zeros((tmdp.nS, tmdp.nA))
ret = 0

a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])

Q = Q_learning(tmdp, s, a, Q, M)

print(Q)
print(get_policy(Q))

[[1768.52212473 1987.17893525]
 [1963.25270449 1624.95773134]
 [1937.31679031 1508.82748886]
 [1891.02653819 1653.37328062]
 [2099.89057812 1693.41518147]
 [1462.20371342 3077.82788508]]
[1, 0, 0, 0, 0, 1]


In [15]:
tmdp_0 = TMDP(river, xi, 0, gamma)
s = tmdp_0.reset()
M = 1000000
Q = np.zeros((tmdp_0.nS, tmdp_0.nA))
ret = 0

a = eps_greedy(s, Q, 1., tmdp_0.allowed_actions[s.item()])

Q = Q_learning(tmdp_0, s, a, Q, M)

print(Q)

[[ 753.62066833 1463.90531677]
 [ 877.75311103 2006.88583723]
 [1605.71172688 3073.57489596]
 [2554.52462858 4388.79396789]
 [3906.9081197  6596.53956963]
 [5323.00627262 9674.65102056]]


In [17]:
pi = get_policy(Q)
tmdp_0 = TMDP(river, xi, 0, gamma)
s = tmdp_0.reset()
M = 1000
Q = np.zeros((tmdp_0.nS, tmdp_0.nA))
ret = 0

a = eps_greedy(s, Q, 1., tmdp_0.allowed_actions[s.item()])

Q = Q_learning(tmdp_0, s, a, Q, M)

print(Q)
print(pi)
print(get_policy(Q))

[[50.         42.30648738]
 [45.         36.507238  ]
 [40.4828096  26.80605896]
 [14.2275224   0.        ]
 [11.91503429  0.        ]
 [ 0.          0.        ]]
[0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0]


In [14]:
print(tmdp.reward)
print(Q)

[[[5.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]]

 [[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]]

 [[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]]

 [[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]]

 [[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]]

 [[0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
  [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 1.e+04]]]


In [11]:
""" taus = [1 - i*0.1 for i in range(10)]
taus.append(0)
Qs = []
xi = np.ones(river.nS)*1/river.nS
gamma = 0.9
river = River(gamma)

for tau in taus:
    tmdp = TMDP(river, xi, tau, gamma)
    s = tmdp.reset()
    M = 10000
    Q = np.zeros((tmdp.nS, tmdp.nA))
    ret = 0
    a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])
    Q = Q_learning(tmdp, s, a, Q, M)
    Qs.append({"tau":tau, "Q_function":Q, "env":tmdp})

for i in range(len(taus)):
    print("Tau:", Qs[i]['tau'])
    print(Qs[i]['Q_function']) """

' taus = [1 - i*0.1 for i in range(10)]\ntaus.append(0)\nQs = []\nxi = np.ones(river.nS)*1/river.nS\ngamma = 0.9\nriver = River(gamma)\n\nfor tau in taus:\n    tmdp = TMDP(river, xi, tau, gamma)\n    s = tmdp.reset()\n    M = 10000\n    Q = np.zeros((tmdp.nS, tmdp.nA))\n    ret = 0\n    a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])\n    Q = Q_learning(tmdp, s, a, Q, M)\n    Qs.append({"tau":tau, "Q_function":Q, "env":tmdp})\n\nfor i in range(len(taus)):\n    print("Tau:", Qs[i][\'tau\'])\n    print(Qs[i][\'Q_function\']) '

In [12]:
""" pi = get_policy(Qs[-2]['Q_function'])
pi_prime = get_policy(Qs[-1]['Q_function'])

print(pi)
print(pi_prime) """

" pi = get_policy(Qs[-2]['Q_function'])\npi_prime = get_policy(Qs[-1]['Q_function'])\n\nprint(pi)\nprint(pi_prime) "

In [19]:
tmdp = TMDP(river, xi, 1, gamma)
Q = np.zeros((tmdp.nS, tmdp.nA))

# Curriculul for decreasing values of tau
for tau in taus:
    tmdp = TMDP(river, xi, tau, gamma)
    s = tmdp.reset()
    M = 10000
    if tau == 0:
        M = 100000
    ret = 0
    a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])
    Q = Q_learning(tmdp, s, a, Q, M)


print("Q function learned from scratch:\n", Qs[-1]['Q_function'])
print("Q function learned with transfer learning:\n", Q)

pi = get_policy(Qs[-1]['Q_function'])
pi_prime = get_policy(Q)

print(pi)
print(pi_prime)


Q function learned from scratch:
 [[50.         43.34466023]
 [45.         38.42221949]
 [40.5        36.17559844]
 [36.44999997 32.32161598]
 [31.04537719  0.        ]
 [ 0.          0.        ]]
Q function learned with transfer learning:
 [[ 605.81142348 1376.80434816]
 [ 825.33973966 1759.68131751]
 [1477.44893237 2800.16732653]
 [2125.69256981 3811.57571264]
 [3743.96004087 6017.86800076]
 [5616.43663002 8931.7819449 ]]
[0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1]
